In [6]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    classification_report,
    accuracy_score,
    f1_score,
    confusion_matrix,
)
from sktime.split import temporal_train_test_split

# Set seaborn style
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams["figure.dpi"] = 100

print("Libraries loaded successfully!")

Libraries loaded successfully!


## Load and Prepare Data

In [7]:
# Data path
DATA_PATH = Path("../../0_data/bissau_merged.nc")

# Center cell indices (for 5x5 grid)
CENTER_LAT_IDX = 2
CENTER_LON_IDX = 2

# Rain threshold (mm)
RAIN_THRESHOLD = 0.1

print(f"Data path: {DATA_PATH}")
print(f"Rain threshold: {RAIN_THRESHOLD} mm")

Data path: ../../0_data/bissau_merged.nc
Rain threshold: 0.1 mm


In [58]:
# Load NetCDF4 with xarray
ds = xr.open_dataset(DATA_PATH)

# Convert timestamps to datetime
time_values = pd.to_datetime(ds.valid_time.values)

# Extract center cell data & apply unit conversions
t2m_center = ds.t2m[:, CENTER_LAT_IDX, CENTER_LON_IDX].values - 273.15  # K → °C
d2m_center = ds.d2m[:, CENTER_LAT_IDX, CENTER_LON_IDX].values - 273.15  # K → °C
tcc_center = ds.tcc[:, CENTER_LAT_IDX, CENTER_LON_IDX].values * 100  # fraction → %
sp_center = ds.sp[:, CENTER_LAT_IDX, CENTER_LON_IDX].values / 100  # Pa → hPa
tp_center = ds.tp[:, CENTER_LAT_IDX, CENTER_LON_IDX].values * 1000  # m → mm

# Create DataFrame with datetime index
df = pd.DataFrame(
    {
        "datetime": time_values,
        "t2m": t2m_center,  # Temperature (°C)
        "d2m": d2m_center,  # Dewpoint (°C)
        "tcc": tcc_center,  # Cloud cover (%)
        "sp": sp_center,  # Surface pressure (hPa)
        "tp": tp_center,  # Precipitation (mm)
    }
)
df.set_index("datetime", inplace=True)


# Calculate Relative Humidity using Magnus formula
def calc_relative_humidity(t2m, d2m):
    a, b = 17.625, 243.04
    rh = 100 * np.exp((a * d2m) / (b + d2m)) / np.exp((a * t2m) / (b + t2m))
    return np.clip(rh, 0, 100)


df["rh"] = calc_relative_humidity(df["t2m"].values, df["d2m"].values)

print(f"Data shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"\nBasic statistics:")
df.head

Data shape: (18263, 6)
Date range: 1975-01-01 00:00:00 to 2024-12-31 00:00:00

Basic statistics:


<bound method NDFrame.head of                   t2m        d2m         tcc           sp   tp         rh
datetime                                                                 
1975-01-01  23.901215  14.001129   31.664080  1008.002686  0.0  53.899139
1975-01-02  23.296295  21.144562   36.313396  1009.499207  0.0  87.722153
1975-01-03  24.607361  20.040924    9.092018  1010.639160  0.0  75.744820
1975-01-04  22.042206  18.102203    0.400352  1010.712708  0.0  78.363899
1975-01-05  21.983063  20.208405  100.000015  1010.250916  0.0  89.676514
...               ...        ...         ...          ...  ...        ...
2024-12-27  23.750641  16.986481    1.199722  1011.753845  0.0  65.858002
2024-12-28  23.277863  13.758942   15.811407  1011.230774  0.0  55.089417
2024-12-29  22.382111  17.828278   88.618683  1011.127502  0.0  75.448097
2024-12-30  22.167755  17.403229    0.000000  1010.276550  0.0  74.417450
2024-12-31  21.788849  18.169098    0.000000  1011.097595  0.0  79.919800

[18263 

## Create Helper Functions

In [ ]:
def get_timeframe(df, month, window_size=1):
    """
    Retrieves the same time period from all years present in the df.
    Use this to avoid searching similar days which are in completely different seasons.
    df: The historical dataframe 
    month: The target month to query
    window_size: How many months each side of the target month to search for
    """
    valid_months = range(month- window_size, month + window_size + 1)
    # Handle wrap-around for Jan/Dec
    valid_months = [m if m >= 0 else 12 for m in valid_months] 
    valid_months = [m if m <= 12 else 1 for m in valid_months]

    if df is None:
        return valid_months

    return df[df.index.month.isin(valid_months)]    

print(get_timeframe(None, 9, 3))
print(get_timeframe(None, 12, 1))
print(get_timeframe(None, 0, 2))

print(get_timeframe(df, 4, 1))

[6, 7, 8, 9, 10, 11, 12]
[11, 12, 1]
[12, 12, 0, 1, 2]
                  t2m        d2m        tcc           sp        tp         rh
datetime                                                                     
1975-03-01  25.868011   9.681549  99.759598  1008.447327  0.000000  36.049957
1975-03-02  23.366302  18.462555  65.093781  1009.395386  0.000000  73.969376
1975-03-03  22.993988  20.936188  82.941643  1011.203308  0.000000  88.203934
1975-03-04  22.538666  19.472076  28.746033  1010.579224  0.000000  82.826927
1975-03-05  22.709015  18.705963  45.916401  1009.494629  0.000000  78.151817
...               ...        ...        ...          ...       ...        ...
2024-05-27  27.469025  24.173981  20.968996  1011.241394  0.000000  82.261566
2024-05-28  27.548096  24.892395  97.372612  1013.597595  0.006865  85.478035
2024-05-29  25.724091  23.913544  98.947441  1014.854675  0.015840  89.754143
2024-05-30  28.496033  24.027252  97.439369  1013.135010  0.000000  76.799973
2024-05-3

In [34]:
import numpy as np
import pandas as pd

def get_knn(df, target_pattern_df, k=5):
    """
    Finds historical periods that match the pattern of multiple columns.
    df: The historical dataframe (cols: Rainfall, Humidity, Temp, etc)
    target_pattern_df: The x-day dataframe of the current situation
    """
    
    # 1. Align Data Types
    # Ensure we are only comparing numbers (floats), no dates/strings
    df_numeric = df.select_dtypes(include=[np.number])
    target_numeric = target_pattern_df.select_dtypes(include=[np.number])
    
    # 2. Prepare Numpy Arrays
    history = df_numeric.values  # Shape: (Total_Days, N_Cols)
    target = target_numeric.values # Shape: (7, N_Cols)
    
    window_len = len(target)
    n_features = history.shape[1]
    
    # 3. Create the "3D View" of History (Instant, no loops)
    # This creates a virtual view of shape: (Rows, 1, Window_Size, N_Features)
    windows = np.lib.stride_tricks.sliding_window_view(
        history, 
        window_shape=(window_len, n_features)
    )
    
    # Squeeze to get Shape: (Rows, Window_Size, N_Features)
    #windows = windows.squeeze()
    windows = windows[:, 0, :, :]
    
    # 4. Calculate MSE (Euclidean Distance)
    # (History_Window - Target_Window)^2
    # We sum errors across the Window (axis 1) and Features (axis 2)
    diff = windows - target
    mse = np.mean(diff ** 2, axis=(1, 2))
    
    # 5. Map results back to Dates
    # The 'mse' array is shorter than the df by (window_len - 1)
    # We use the index corresponding to the END of each window
    result_index = df.index[window_len-1:]
    
    results = pd.Series(mse, index=result_index, name='similarity_score')
    
    return results.nsmallest(k)

# --- USAGE ---
# 1. Define your target (e.g., the first 7 days of the dataset)
target_days = df.iloc[0:7] 

# 2. Run the search
# It automatically compares Rainfall vs Rainfall, Humidity vs Humidity, etc.
best_matches = get_knn(df.copy(), target_days)

print("Target days: ", target_days)

print("--- Top 5 Similar Historical Periods ---")
print(best_matches)



Target days:                    t2m        d2m         tcc           sp   tp         rh
datetime                                                                 
1975-01-01  23.901215  14.001129   31.664080  1008.002686  0.0  53.899139
1975-01-02  23.296295  21.144562   36.313396  1009.499207  0.0  87.722153
1975-01-03  24.607361  20.040924    9.092018  1010.639160  0.0  75.744820
1975-01-04  22.042206  18.102203    0.400352  1010.712708  0.0  78.363899
1975-01-05  21.983063  20.208405  100.000015  1010.250916  0.0  89.676514
1975-01-06  22.323212  20.448883    7.034683  1010.767029  0.0  89.151909
1975-01-07  21.732574  18.764679    0.559998  1012.075684  0.0  83.243317
--- Top 5 Similar Historical Periods ---
datetime
1975-01-07     0.000000
2019-05-12    36.506321
1981-04-20    42.278000
1998-05-31    43.709774
1993-04-12    44.558174
Name: similarity_score, dtype: float32


## Generate Test Set

In [59]:
y = pd.DataFrame()
y["rain_binary"] = (df["tp"] > RAIN_THRESHOLD).astype(int)
df = df.drop(columns=['tp'])

X_train = df.loc["1976-01-16": "2020-01-25"]
y_train =y.loc["1976-01-16": "2020-01-25"]

X_test = df.loc["2020-01-26": "2024-12-17"]
y_test = y.loc["2020-01-26": "2024-12-17"]

X_train

,t2m,d2m,tcc,sp,rh
datetime,,,,,
1976-01-16,23.344452,9.829498,3.069782,1011.577820,42.339874
1976-01-17,22.269806,7.887360,99.118134,1011.335571,39.636539
1976-01-18,21.866119,7.746979,100.000015,1009.373047,40.237514
1976-01-19,19.794952,13.104645,94.595528,1008.343506,65.332451
1976-01-20,21.123810,16.276764,31.339640,1010.161865,73.852829
...,...,...,...,...,...
2020-01-21,23.705719,20.141388,97.974464,1009.564697,80.453857
2020-01-22,24.855560,18.866119,96.625290,1011.302246,69.373589
2020-01-23,24.614746,18.754425,92.453003,1011.209290,69.889435


In [ ]:
probabililty_score = lambda array: np.sum(array)/len(array)
majority_score = lambda array: np.bincount(array).argmax()


def get_prediction(x, X_train, y_train, score_function):
    valid_months = get_timeframe(X_train, x.index.month[0])
    best_matches = get_knn(valid_months, x)
    rainy_days = y_train.loc[best_matches.index.to_list()]["rain_binary"].to_list()
    
    return score_function(rainy_days)

get_prediction(X_test.iloc[0:1], X_train, y_train, probabililty_score)


0.0

In [60]:
# Single day compared to training set
y_test_pred = X_test.apply(lambda x: get_prediction(x.to_frame().T, X_train, y_train, majority_score), axis=1)
y_test_prob = X_test.apply(lambda x: get_prediction(x.to_frame().T, X_train, y_train, probabililty_score), axis=1)


test_acc = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_prob)
print("Accuracy is ", test_acc)
print("F1 score is ", test_f1)
print("ROC AUC is ", test_auc)


Accuracy is  0.8920581655480985
F1 score is  0.4469914040114613
ROC AUC is  0.8226391737391363


In [ ]:
import pandas as pd

# 1. Prepare "Context Data"
# We attach the last 7 days of X_train to the START of X_test.
# This ensures the very first row of X_test has a valid 7-day history window.
# Min value is 1
window_size = 7

# We only need (window_size - 1) past days to complete the first window.
if window_size > 1:
    # Take the last (N-1) rows from training
    history_padding = X_train.iloc[-(window_size - 1):]
    context_data = pd.concat([history_padding, X_test])
else:
    # If window is 1, we don't need any history from Train
    context_data = X_test

predictions = []
predictions_prob = []


# 2. Iterate strictly over the X_test portion
for i in range(len(X_test)):
    
    # The 'end' of our window is the current step in the test set
    # The 'start' is shifted back by window_size
    # We add 'window_size' to i because we padded the start of the array
    end_idx = i + window_size
    start_idx = i 
    
    # Extract the 7-day window ending just before the target day
    # (Or including it, depending on your specific logic)
    current_window = context_data.iloc[start_idx : end_idx]
    
    # Make the prediction by passing through the window
    pred = get_prediction(current_window, X_train, y_train, majority_score)
    predictions.append(pred)

    pred_prob = get_prediction(current_window, X_train, y_train, probabililty_score)
    predictions_prob.append(pred_prob)

# 3. Create Result Series
y_test_pred = pd.Series(predictions, index=X_test.index)
y_test_prob = pd.Series(predictions_prob, index=X_test.index)


print("Predictions complete.")
test_acc = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_prob)
print("Accuracy is", test_acc)
print("F1 score is", test_f1)
print("ROC AUC is", test_auc)


Predictions complete.
Accuracy is  0.8831096196868009
F1 score is  0.43360433604336046
ROC AUC is  0.8535545533862872


1 day window:
Predictions complete.
Accuracy is  0.8920581655480985
F1 score is  0.4469914040114613
ROC AUC is  0.8226391737391363

3 day window:
Predictions complete.
Accuracy is  0.8853467561521253
F1 score is  0.43526170798898073
ROC AUC is  0.8457077946358764

7 day window: 
Predictions complete.
Accuracy is  0.8831096196868009
F1 score is  0.43360433604336046
ROC AUC is  0.8535545533862872